In [7]:
import open3d as o3d
import numpy as np
from sympy import *

### Symbolic functions

In [8]:
def Ry_sym(theta):
    ct = cos(theta)
    st = sin(theta)
    R = Matrix([[ct, 0.0, st], [0.0, 1.0, 0.0], [-st, 0, ct]])
    return R

def Rx_sym(theta):
    ct = cos(theta)
    st = sin(theta)
    R = Matrix([[1.0, 0.0, 0.0], [0.0, ct, -st], [0.0, st, ct]])
    return R

## Task 2.5

In [9]:
class RobotObject:
    robotObjects = [o3d.geometry.TriangleMesh.create_coordinate_frame(size=75)] #List of all o3d objects, initialized with {s}-frame

    def __init__(self, num_joints, length_links, link_orient = 'x'):
        self.num_joints = num_joints
        self.length_links = length_links
        self.link_orient = ['x']*(len(link_lengths)+1) if link_orient == 'x' else link_orient
        self.joints = []
        self.links = []
        self.make_robot_objects()

#creates all o3d-objects of the robot in {s}
    def make_robot_objects(self): 
        for i in range(self.num_joints):
            self.joints.append(Joint())
        for i in range(len(self.length_links)):
            self.links.append(Link(self.length_links[i], self.link_orient[i]))

#Moves all objects from {s} to config given by T_list
    def transform(self, T_list):
        for i, J in enumerate(self.joints):
            J.transform(T_list[i])
        for i, L in enumerate(self.links):
            L.transform(T_list[i])

    def draw_robot(self):
        o3d.visualization.draw_geometries(self.robotObjects)

#_____________Joint Class______________#
class Joint(RobotObject):

    def __init__(self):
        self.joint = o3d.geometry.TriangleMesh.create_cylinder(
            radius=10, height=30)
        self.coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50)
        self.update_mesh_list()
        self.set_colour()

    def update_mesh_list(self):
        self.robotObjects.append(self.joint)
        self.robotObjects.append(self.coord)

    def set_colour(self, colour=[0, 1, 0]):
        self.joint.paint_uniform_color(colour)

    def transform(self, T):
        self.joint = self.joint.transform(T)
        self.coord = self.coord.transform(T)

#____________Link Class_________________#
class Link(RobotObject):

    def __init__(self, lenght, orient):
        self.lenght = lenght
        self.link_orient = orient

        if (orient == 'x'): ##Needed for defining link axis in DH
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).rotate(Ry_sym(np.pi/2)).translate(np.array([self.lenght/2, 0, 0]))
        elif (orient == 'y'):
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).rotate(Rx_sym(-np.pi/2)).translate(np.array([0, self.lenght/2, 0]))
        elif (orient == 'z'):
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).translate(np.array([0, 0, self.lenght/2]))
        
        self.update_mesh_list()
        self.set_colour()

    def update_mesh_list(self):
        self.robotObjects.append(self.link)

    def set_colour(self, colour=[0, 0, 1]):
        self.link.paint_uniform_color(colour)

    def transform(self, T):
        self.link = self.link.transform(T)

In [10]:
def make_T_list(Mi, S_list, thetas):
    T_list = []  # List with T01,T02,T03...
    T = np.eye(4)
    for i in range(len(thetas)):
        T = T @ exp6(S_list[:, i], thetas[i]) @ Mi[:, 4*i:4*(i+1)]
        T_list.append(T)
    return T_list



In [11]:
link_lengths = np.array([0.1,455,35,420,80])
# link_lengths = np.array([150, 150, 150, 150, 150])

robo = RobotObject(6, link_lengths,['x','x','x','z','x'])
T_list = make_T_list(Mi, S_sp, [0, 0, 0, 0, 0, 0])

robo.transform(T_list)
robo.Joints[0].set_colour([1, 0, 0])
robo.Joints[5].set_colour([0, 0, 1])
T_list
robo.draw_robot()
l = ['z']*(len(link_lengths)+1)
link_lengths

NameError: name 'Mi' is not defined